### Email extractions from Gmail

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client tiktoken

In [5]:
import os
import pickle
import json
import base64
import google.auth.transport.requests
import google_auth_oauthlib.flow
import googleapiclient.discovery
from google.oauth2 import service_account


# Constants
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]
CLIENT_SECRET_FILE = "client_secret_gmail.json"
TOKEN_FILE = "token.pickle"
SERVICE_ACCOUNT_JSON = "service_account.json"

# Authentication function for Gmail API
def authenticate_gmail():
    creds = None
    # Check if token file exists and load credentials
    if os.path.exists(TOKEN_FILE):
        with open(TOKEN_FILE, "rb") as token:
            creds = pickle.load(token)
    # If credentials are not valid or don't exist, refresh or create new ones
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(google.auth.transport.requests.Request())
        else:
            flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save new credentials to token file
        with open(TOKEN_FILE, "wb") as token:
            pickle.dump(creds, token)
    # Build and return Gmail API service
    return googleapiclient.discovery.build("gmail", "v1", credentials=creds)

# Function to fetch emails from a specific sender and save their contents
def get_emails(service, sender, max_results):
    # List messages from the sender
    results = service.users().messages().list(userId="me", q=f"from:{sender}", maxResults=max_results).execute()
    messages = results.get("messages", [])
    email_contents = []

    # Iterate through messages and extract subject and body
    for message in messages:
        msg = service.users().messages().get(userId="me", id=message["id"]).execute()
        msg_payload = msg.get("payload")
        headers = msg_payload.get("headers")
        parts = msg_payload.get("parts")
        data = ""

        # Find the subject of the email
        for header in headers:
            if header.get("name").lower() == "subject":
                subject = header.get("value")

        # Extract the plain text body of the email
        if parts:
            for part in parts:
                if part["mimeType"] == "text/plain":
                    data = base64.urlsafe_b64decode(part["body"]["data"]).decode()

        # Append subject and body to email_contents list
        email_contents.append({"subject": subject, "body": data})

    # Return the list of email contents
    return email_contents

# Function to save email contents to a JSON file
def save_to_json(contents, filename="emails.json"):
    with open(filename, "w") as outfile:
        json.dump(contents, outfile, indent=4)

In [6]:
# Autenticazione per Gmail
gmail_service = authenticate_gmail()
emails = get_emails(gmail_service, "ainews@buttondown.email", 5)
save_to_json(emails)
print("Emails saved to emails.json")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=472431379805-0fk314jqpf22u86f18pcq9ub1974qhmc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A40051%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=bXS8BoL1fXju93gLoV8httJmEiae3O&access_type=offline
Emails saved to emails.json


In [13]:
# analyze the email.json file
import json
import re
import tiktoken

# tiktoken
enc = tiktoken.encoding_for_model("gpt-4o")
# Load the email contents from the JSON file
with open("emails.json", "r") as file:
    emails = json.load(file)
emails
# length of for each element in the json file, in tokens
for email in emails:
    email["length"] = len(enc.encode(email["body"]))
# average length of the emails
total_length = sum(email["length"] for email in emails)
average_length = total_length / len(emails)
average_length


13829.8

# Chiamata a Vertex

In [ ]:
# Authenticates for Vertex AI
def authenticate_vertex_ai():
    credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_JSON)
    aiplatform.init(credentials=credentials)

# Example Vertex AI function
def example_vertex_ai_function():
    # Assumiamo che tu abbia già autenticato aiplatform.init()
    # qui puoi implementare la tua logica per chiamare le API Vertex AI
    print("Calling Vertex AI with authenticated credentials...")
    # Esempio di codice per fare una previsione o altro:
    # prediction_client = aiplatform.gapic.PredictionServiceClient()
    # ... o altra logica di Vertex AI

In [ ]:
# Autenticazione per Vertex AI
authenticate_vertex_ai()
example_vertex_ai_function()